### Full POP model works

In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

and as dissipator is traceless:

$
A = \begin{pmatrix} a_1 + i b_1 &   a_2 + i b_2\\  a_3 + i b_3 & -a_1 - i b_1
   \end{pmatrix} $
   
$\operatorname{Tr} A = 0$

3. The Lindblad equation is also invariant under any unitary transformation $v$ of Lindblad operators and constants:

$\sqrt{\gamma_i}A_i \rightarrow \sqrt{\gamma'_i}A'_i = \sum_j v_{ij} \sqrt{\gamma} A_j$

if we have just one operator $A$ it means that we can pick any of its unitary invariants.

   
So the $A$ operator is both traceless and phase invariant
$
A = \begin{pmatrix} a e^{i \phi_a} & b e^{i \phi_b} \\ c e^{i \phi_c} & -a e^{i \phi_a}
   \end{pmatrix} 
= \begin{pmatrix} a e^{i (\phi_a - \phi_b)} & b \\ c e^{i (\phi_c - \phi_b)} & -a e^{i (\phi_a - \phi_b)}
   \end{pmatrix} e^{i \phi_b}
$

Therefore we can possibly look for jump operator of the form:

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12} \\ a_{21} + i b_{21} & -a_{11} - i b_{11} 
   \end{pmatrix} 
$

or

$
A = \begin{pmatrix} a_{11} & a_{12} + i b_{12}  \\ a_{21} + i b_{21} & -a_{11}
   \end{pmatrix} 
$

In [2]:
#using DynamicPolynomials

@polyvar g

@polyvar e[1]
@polyvar h[1:2]

#Hˢʸᵐᵇ = [ 0.      0.
#         0.     e[1] ]

# Hˢʸᵐᵇₐₙ
Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ]

In [3]:
@polyvar a[1:4]
@polyvar b[1:4]

#Aˢʸᵐᵇ = [  0.           a[2] + im*b[2]
#           0.           0.             ]

Aˢʸᵐᵇ = [ a[1]+im*b[1]   a[2]
          0               -a[1]-im*b[1] ]

In [4]:
pwd()

In [5]:
parentdir = pwd()
data_dir = parentdir*"/DATA/"
println(data_dir)

output_directory = parentdir*"/MODELS/"

### Train models loop

In [6]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
res_file_name = "POP_train-B-zero-down-off-diag_" * date_and_time_string * ".h5"

#α = 0. #regularization coefficient

println(res_file_name)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in γ

    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
    
    start_time = time()
    
    objₑₓ = 0
    #objₑₓₐₙ = 0

    for (ρ,t) in zip([ρᵍ, ρᵉ, ρˣ, ρʸ], [tᵍ, tᵉ, tˣ, tʸ])
        
        t = convert(Vector{Float64}, t)
        @assert maximum(diff(t)) ≈ minimum(diff(t)) ≈ t[2]-t[1]

        # Convert cut ρ series:
        ρ = convert(Vector{Matrix{ComplexF64}}, ρ)
        objₑₓ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
        #objₑₓₐₙ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇₐₙ, [Aˢʸᵐᵇ], g)
        
    end # of files (initial states) loop
    
    #objₑₓ += α * (sum(a.^4)+sum(b.^4)+sum(h.^2)) #adding regularization
    #objₑₓ += α *  sum(variables(objₑₓ).^3)
    
    #objₑₓ += α * (sum(a.^2)+sum(b.^2)) #adding regularization
    
    #objₑₓ += α *sum(variables(tr(Aˢʸᵐᵇ*Aˢʸᵐᵇ')).^2)
      
    solₑₓⁿᵉʷ, best_methodₑₓⁿᵉʷ = LiPoSID.sos_min_newton(objₑₓ) 
    
    runtime = time() - start_time
    
    Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, solₑₓⁿᵉʷ)
    Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, solₑₓⁿᵉʷ)
    
    #gˢⁱᵈₑₓ = subs(g, solₑₓⁿᵉʷ)
    
    
    h5open(output_directory*res_file_name,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)

        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈₑₓ)
        γ_group["A"] = convert.(ComplexF64, Aˢⁱᵈₑₓ)
        
        #γ_group["g"] = convert(Float64, gˢⁱᵈₑₓ)
        
        γ_group["runtime"] = runtime
        
    end # of HDF5 writing


end